In [6]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

In [1]:
geometry = {
    "xmin": -77.0036,
    "ymin": 38.8879,
    "xmax": -76.98434,
    "ymax": 38.90007,
    "spatialReference": {
        "wkid":4326
    }
    
}
geometryType = 'esriGeometryEnvelope'

In [89]:
import os
from collections.abc import Generator

from arcgis.geometry import Polygon, Envelope
from azure.cosmos import CosmosClient, ContainerProxy
import h3

In [7]:
cosmos_url = os.getenv('COSMOS_URL')
cosmos_key = os.getenv('COSMOS_KEY')
cosmos_db = os.getenv('COSMOS_DB')
cosmos_container = os.getenv('COSMOS_CONTAINER')

client = CosmosClient(url=cosmos_url, credential=cosmos_key)
db = client.get_database_client('Yelp')
container = db.get_container_client('Items')

container

<ContainerProxy [dbs/Yelp/colls/Items]>

In [13]:
h3_bndry = h3.cell_to_boundary('8828d5911dfffff')

In [25]:
from arcgis.geometry import Polygon

In [44]:
coord_lst = list((x, y) for y, x in h3_bndry)
coord_lst.append(coord_lst[0])
coord_lst.reverse()

poly = Polygon({'rings': [coord_lst], 'spatialReference': {'wkid': 4326}})

xmin = poly.envelope.xmin
xmax = poly.envelope.xmax
ymin = poly.envelope.ymin
ymax = poly.envelope.ymax

xmin, xmax, ymin, ymax

(-122.91406047458825,
 -122.90102650934975,
 47.03816624517227,
 47.04654317996362)

In [108]:
def get_intersecting_bounding_coordinates(container: ContainerProxy, xmin: float, xmax: float, ymin: float, ymax: float) -> Generator[dict]:

    # create a GeoJSON bounding box for the query
    bbox = {'type': 'Polygon', 'coordinates': [[[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin], [xmin, ymin]]]}

    # since query items is a generator, piggyback on this paradigm to generate results on-demand
    for itm in container.query_items(f'SELECT * FROM c WHERE ST_INTERSECTS(c.geometry, {bbox})', enable_cross_partition_query=True):
        yield itm
        

def get_intersecting_envelope(container: ContainerProxy, envelope: Envelope) -> Generator[dict]:
    
    # wrap the function taking raw bounding coordinates to get features in envelope
    itm_gen =  get_intersecting_bounding_coordinates(container, envelope.xmin, envelope.xmax, envelope.ymin, envelope.ymax)
    
    return itm_gen

def get_intersecting_polygon(container: ContainerProxy, polygon: Polygon) -> Generator[dict]:
    
    # get the geojson representation of the polygon
    geojson_poly = polygon.__geo_interface__
    
    # ensure not multipart geometry
    if len(geojson_poly['coordinates'][0]) > 1:
        raise ValueError('Cannot process multipart polygons. Please separate into single part polygons.')
    
    # convert to polygon with coordinates as proper lists (not tuples) so it can be processed by cosmos
    geojson_poly['type'] = 'Polygon'
    geojson_poly['coordinates'] = [[list(coords) for coords in geojson_poly['coordinates'][0][0]]]
    
    # since query items is a generator, piggyback on this paradigm to generate results on-demand
    for itm in container.query_items(f'SELECT * FROM c WHERE ST_INTERSECTS(c.geometry, {geojson_poly})', enable_cross_partition_query=True):
        yield itm
        
        
list(get_intersecting_polygon(container, poly))

[{'id': 'EpXCy_SSO-x8BjRLc_xUcg',
  'alias': 'the-westside-tavern-olympia',
  'name': 'The Westside Tavern',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/Ms0bt1riRWKO7PR4sNAV3w/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/the-westside-tavern-olympia?adjust_creative=ZAyA5LhP0CVLQ3sJwFX2SA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ZAyA5LhP0CVLQ3sJwFX2SA',
  'review_count': 211,
  'categories': [{'alias': 'bars', 'title': 'Bars'},
   {'alias': 'burgers', 'title': 'Burgers'},
   {'alias': 'newamerican', 'title': 'New American'}],
  'rating': 4.0,
  'coordinates': {'latitude': 47.0457916, 'longitude': -122.9243317},
  'transactions': ['delivery'],
  'price': '$$',
  'location': {'address1': '1815 Harrison Ave NW',
   'address2': '',
   'address3': '',
   'city': 'Olympia',
   'zip_code': '98502',
   'country': 'US',
   'state': 'WA',
   'display_address': ['1815 Harrison Ave NW', 'Olympia, WA 98502']},
  'phone': '+13609157839',
  '

In [10]:
[itm for itm in container.query_items("SELECT * FROM c WHERE c.h3_09 = '8928d5911cbffff'", enable_cross_partition_query=True)]

[{'id': 'RucjyhLr6QiHdlQH1eKaoA',
  'name': 'Cascadia Grill',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/cZjjKcIgnerbWUeZ2M6CNA/o.jpg',
  'url': 'https://www.yelp.com/biz/cascadia-grill-olympia-3?adjust_creative=ZAyA5LhP0CVLQ3sJwFX2SA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ZAyA5LhP0CVLQ3sJwFX2SA',
  'review_count': 554,
  'categories': [{'alias': 'steak', 'title': 'Steakhouses'},
   {'alias': 'bars', 'title': 'Bars'},
   {'alias': 'brasseries', 'title': 'Brasseries'}],
  'rating': 4.5,
  'price': '$$',
  'h3_07': '8728d5911ffffff',
  'h3_08': '8828d5911dfffff',
  'h3_09': '8928d5911cbffff',
  'h3_10': '8a28d5911c9ffff',
  'h3_11': '8b28d5911c9cfff',
  'geometry': {'type': 'Point',
   'coordinates': [-122.90313455582186, 47.04489344348968]},
  'address': '200 4th Ave W, Olympia, WA 98501',
  'retrieval_timestamp': '2024-01-02T14:35:41.990521',
  '_rid': 'l1clAJGNYXYEAAAAAAAAAA==',
  '_self': 'dbs/l1clAA==/colls/l1clAJGNYXY=/docs/l1clAJGNYXYEAA